In [ ]:
import sklearn
from sklearn import datasets
from sklearn.metrics import silhouette_score
import numpy as np 
import matplotlib.pyplot as plt 
import copy 

In [ ]:
# Step 1: Load the Digit dataset
digits = datasets.load_digits(n_class=5)
X = digits.data
print(X.shape)

In [ ]:
# Step 2: euclidean distance 
def euclidean_distances(A, B):
    n, d = A.shape
    m, d1 = B.shape
    assert d == d1, 'Incompatible shape'
    A_squared = np.sum(np.square(A), axis=1, keepdims=True)
    B_squared = np.sum(np.square(B), axis=1, keepdims=True)
    AB = np.matmul(A, B.T)
    distances = np.sqrt(A_squared - 2 * AB + B_squared.T)
    return distances

In [ ]:
# Step 3: find eps-neighborhood of a point
def find_eps_neighborhood(distances, ind, eps):
    ''' 
    Input arguments:
        - distances: a matrix containing distances between all pairs of points in the dataset
        - ind: index of the point of interest
        - eps: the epsilon parameter
    Output:
        - Retun a set of points in the neighborhood. 
        (Note: Use the 'set' data structure in Python)
    '''
    ### YOUR CODE HERE ###

In [ ]:
# Step 4: find all reachable points of a given point w.r.t eps 
def find_reachable_pts(distances, eps, ind):
    eps_neighbors = find_eps_neighborhood(distances, ind, eps)
    reachables = eps_neighbors
    new_pts = copy.deepcopy(eps_neighbors)
    new_pts.remove(ind)
    while len(new_pts) > 0:
        pt = new_pts.pop() 
        pt_neighbors = find_eps_neighborhood(distances, pt, eps)
        additional_pts = pt_neighbors.difference(reachables)
        reachables.update(additional_pts)
        new_pts.update(additional_pts)
    return reachables

In [ ]:
def dbscan(X, eps, minPts):
    ''' a simple implementation of DBSCAN algorithm
    In this implementation, a point is represented by its index in the dataset. 
    In this function, except for the step to calculate the Euclidean distance,
    we will only work with the points't indices.
    
    Input arguments:
        - X: the dataset
        - eps: the epsilon parameter
        - minPts: the minimum number of points for a cluster
    Output:
        - core_points: a list containing the indices of the core points
        - cluster_labels: a Numpy array containing labels for each point in X
        - outliers: a list containing the indices of the outlier points
    '''
    # a list to keep track of the unvisited points
    unvisited = list(range(X.shape[0]))
    # list of core points (or cluster centroids)
    core_points = list([])
    # list of clusters, each cluster is a set of points
    clusters = list([])
    # list of outlier points (or noises)
    outliers = list([])
    distances = euclidean_distances(X, X)
    
    while True:
        # randomly choose a point, p, from the list of unvisited points
        ind = ### YOUR CODE HERE ###
        
        # find the eps-neighborhood of the chosen point p
        eps_neighbors = ### YOUR CODE HERE ###
        
        # check if p is a core point or not
        is_core_pt =  ### YOUR CODE HERE ###
        
        if is_core_pt:
            # add the chosen index to the core_points list
            ### YOUR CODE HERE ###
            
            
            # find all reachable points from p w.r.t eps and form a new cluster
            new_cluster = find_reachable_pts(distances, eps, ind) ### YOUR CODE HERE ###
            
            # add the newly formed cluster to the list of cluster
            ### YOUR CODE HERE ###
            
            
            # remove the indices in the new_cluster from the unvisited list and the outlier list,
            # if they were added to either of those list before
            ### YOUR CODE HERE ###
            
            
        else:
            # if not core point, add p to the list of outlier points
            ### YOUR CODE HERE ###
            
        
        # remove the chosen index from the unvisited list (if it is still inside this list)
        ### YOUR CODE HERE ###

            
        # if there is no point left in the unvisited list, stop the loop
        if len(unvisited) == 0:
            break
    
    # convert the resulting cluster list to cluster_labels
    cluster_labels = np.zeros(X.shape[0])
    for i in range(len(clusters)):
        for j in clusters[i]:
            cluster_labels[j] = i

    return core_points, cluster_labels, outliers

In [ ]:
eps = 20.0
minPts = 10
core_points, cluster_labels, outliers = dbscan(X, eps, minPts)
print('%d clusters found' %(len(core_points)))
print('%d outlier points detected' %(len(outliers)))

# visualize the clustering result
selected_cluster = 1
X_cluster_1 = X[cluster_labels == selected_cluster]
n_img_per_row = 10
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img_indx = i * n_img_per_row + j
        if img_indx < len(X_cluster_1):
            img[ix:ix + 8, iy:iy + 8] = X_cluster_1[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title('A selection of 100 images from cluster {:}'.format(selected_cluster))
plt.show()

# Calculate the shlhouette score
if len(core_points) > 1:
    print('Silhouette score: %f' %silhouette_score(X, cluster_labels))
else:
    print('Cannot evaluate silhouetter score with only one cluster')